In [4]:
import sys
import cv2
import numpy as np
from keras.models import load_model
import mediapipe as mp
from PyQt5.QtCore import QSettings

# Load the trained model
model = load_model('gesture_recognition_model.h5')

# Initialize Mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

# State variables
mode = 0

def switch_mode():
    global mode
    mode += 1
    print(f"Switched to mode {mode}")

# Load settings
settings = QSettings("MyApp", "MyAppSettings")
camera_str = settings.value("camera", "Camera 0")

# Extract the camera index from the string
camera_index = int(camera_str.split()[-1])

# Open the webcam with the camera index from settings
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"Error: Could not open camera with index {camera_index}")
    sys.exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Prepare the landmarks for prediction
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            flattened_landmarks = np.array([coord for point in landmarks for coord in point]).reshape(1, -1)

            # Predict the gesture
            prediction = model.predict(flattened_landmarks)
            predicted_mode = np.argmax(prediction)

            cv2.putText(frame, f'Mode: {predicted_mode}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Hand Gesture Recognition', frame)

    key = cv2.waitKey(1)
    if key == ord('2'):
        switch_mode()
    elif key == ord('3'):
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'keras'